In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.utils import resample
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score

from imblearn.over_sampling import SMOTE

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler



from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score, roc_curve, auc
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn import tree 
from sklearn.preprocessing import LabelEncoder
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics

In [32]:
url = "https://data.cityofnewyork.us/resource/fhrw-4uyv.json?$where=created_date%20between%20%272019-04-11T00:00:00%27%20and%20%272019-04-12T00:00:00%27"
data = pd.read_json(url)

In [43]:
data.tail()

,address_type,agency,agency_name,bbl,borough,bridge_highway_direction,bridge_highway_name,bridge_highway_segment,city,closed_date,...,park_facility_name,resolution_action_updated_date,resolution_description,road_ramp,status,street_name,taxi_pick_up_location,unique_key,x_coordinate_state_plane,y_coordinate_state_plane
995,ADDRESS,HPD,Department of Housing Preservation and Develop...,4.015110e+09,QUEENS,NaN,NaN,NaN,Elmhurst,2019-04-13T02:13:10.000,...,Unspecified,2019-04-13T02:13:10.000,The complaint you filed is a duplicate of a co...,NaN,Closed,ELMHURST AVENUE,NaN,42197021,1018372.0,211606.0
996,ADDRESS,HPD,Department of Housing Preservation and Develop...,4.015110e+09,QUEENS,NaN,NaN,NaN,Elmhurst,2019-04-13T02:13:10.000,...,Unspecified,2019-04-13T02:13:10.000,The complaint you filed is a duplicate of a co...,NaN,Closed,ELMHURST AVENUE,NaN,42198022,1018372.0,211606.0
997,ADDRESS,HPD,Department of Housing Preservation and Develop...,4.015110e+09,QUEENS,NaN,NaN,NaN,Elmhurst,2019-04-13T02:13:10.000,...,Unspecified,2019-04-13T02:13:10.000,The complaint you filed is a duplicate of a co...,NaN,Closed,ELMHURST AVENUE,NaN,42193360,1018372.0,211606.0
998,ADDRESS,DOT,Department of Transportation,4.103470e+09,QUEENS,NaN,NaN,NaN,SAINT ALBANS,2019-05-02T10:29:27.000,...,Unspecified,2019-05-02T10:29:27.000,The Department of Transportation inspected the...,NaN,Closed,DUNLOP AVENUE,NaN,42200621,1047247.0,195180.0
999,ADDRESS,NYPD,New York City Police Department,3.065660e+09,BROOKLYN,NaN,NaN,NaN,BROOKLYN,2019-04-12T05:03:12.000,...,Unspecified,2019-04-12T05:03:12.000,The Police Department responded and upon arriv...,NaN,Closed,AVENUE M,NaN,42193552,991995.0,163948.0


In [34]:
data.columns

Index(['address_type', 'agency', 'agency_name', 'bbl', 'borough',
       'bridge_highway_direction', 'bridge_highway_name',
       'bridge_highway_segment', 'city', 'closed_date', 'community_board',
       'complaint_type', 'created_date', 'cross_street_1', 'cross_street_2',
       'descriptor', 'due_date', 'facility_type', 'incident_address',
       'incident_zip', 'intersection_street_1', 'intersection_street_2',
       'landmark', 'latitude', 'location', 'location_type', 'longitude',
       'open_data_channel_type', 'park_borough', 'park_facility_name',
       'resolution_action_updated_date', 'resolution_description', 'road_ramp',
       'status', 'street_name', 'taxi_pick_up_location', 'unique_key',
       'x_coordinate_state_plane', 'y_coordinate_state_plane'],
      dtype='object')

In [35]:
data.complaint_type.value_counts()

HEAT/HOT WATER                         146
Illegal Parking                        111
Noise - Residential                     95
Rodent                                  95
Blocked Driveway                        81
Noise                                   77
Street Condition                        40
Noise - Street/Sidewalk                 27
Noise - Vehicle                         19
Noise - Commercial                      18
Sewer                                   15
Sanitation Condition                    14
Water System                            13
General Construction/Plumbing           13
Request Large Bulky Item Collection     13
Derelict Vehicle                        13
UNSANITARY CONDITION                    12
Construction Safety Enforcement         11
Unsanitary Animal Pvt Property          11
PAINT/PLASTER                           10
Dirty Conditions                         9
Food Poisoning                           9
Street Light Condition                   8
PLUMBING   

In [36]:
data.borough.value_counts()

BROOKLYN         307
MANHATTAN        237
QUEENS           225
BRONX            180
STATEN ISLAND     47
Unspecified        4
Name: borough, dtype: int64

In [19]:
test = data.groupby(['borough', 'complaint_type'])

In [37]:
data['complaint_type'][data['borough']=='BROOKLYN'].value_counts()

Illegal Parking                        45
Rodent                                 39
HEAT/HOT WATER                         33
Noise - Residential                    29
Blocked Driveway                       24
Street Condition                       13
Noise                                  13
Noise - Commercial                      8
General Construction/Plumbing           7
Water System                            6
PAINT/PLASTER                           6
Sanitation Condition                    5
Sewer                                   4
Noise - Helicopter                      4
Derelict Vehicle                        4
Unsanitary Animal Pvt Property          4
Derelict Vehicles                       4
UNSANITARY CONDITION                    4
FLOORING/STAIRS                         4
Noise - Street/Sidewalk                 4
Noise - Vehicle                         3
PLUMBING                                3
Air Quality                             3
Dirty Conditions                  

In [38]:
data['complaint_type'][data['borough']=='QUEENS'].value_counts()


Blocked Driveway                       35
Illegal Parking                        34
Rodent                                 16
Noise - Residential                    15
HEAT/HOT WATER                         14
Noise                                  14
Request Large Bulky Item Collection    11
Street Condition                        8
Sanitation Condition                    6
Derelict Vehicle                        6
Noise - Vehicle                         6
Noise - Street/Sidewalk                 5
Building/Use                            4
Food Poisoning                          4
Noise - Commercial                      4
General Construction/Plumbing           4
Derelict Vehicles                       3
Street Sign - Missing                   3
Graffiti                                3
ELECTRIC                                2
Water System                            2
Elevator                                2
Air Quality                             2
Unsanitary Animal Pvt Property    

In [39]:
data['complaint_type'][data['borough']=='MANHATTAN'].value_counts()


Noise                                  46
HEAT/HOT WATER                         41
Noise - Residential                    22
Rodent                                 18
Noise - Street/Sidewalk                15
Construction Safety Enforcement        11
Illegal Parking                         8
Street Condition                        7
Noise - Vehicle                         7
Noise - Commercial                      6
Sewer                                   5
PLUMBING                                4
Dirty Conditions                        3
UNSANITARY CONDITION                    3
PAINT/PLASTER                           3
Food Poisoning                          3
Homeless Person Assistance              3
For Hire Vehicle Complaint              2
Water System                            2
DOOR/WINDOW                             2
Unsanitary Animal Pvt Property          2
WATER LEAK                              2
Blocked Driveway                        2
GENERAL                           

In [42]:
data['complaint_type'][data['borough']=='BRONX'].value_counts()


HEAT/HOT WATER                    58
Noise - Residential               27
Rodent                            20
Blocked Driveway                  15
Illegal Parking                   14
Street Condition                   6
UNSANITARY CONDITION               4
Sewer                              3
Noise                              3
Elevator                           3
Noise - Street/Sidewalk            3
Sanitation Condition               3
Street Light Condition             2
Dirty Conditions                   2
Derelict Vehicle                   2
Unsanitary Animal Pvt Property     2
PLUMBING                           1
Standing Water                     1
GENERAL                            1
Animal Abuse                       1
Noise - Vehicle                    1
Sidewalk Condition                 1
Hazardous Materials                1
Food Poisoning                     1
ELECTRIC                           1
Traffic                            1
New Tree Request                   1
W

In [41]:
data['complaint_type'][data['borough']=='STATEN ISLAND'].value_counts()


Illegal Parking                      10
Street Condition                      6
Blocked Driveway                      5
Water System                          3
Standing Water                        2
Sewer                                 2
Rodent                                2
WATER LEAK                            2
Noise - Vehicle                       2
Noise - Residential                   2
Electronics Waste Appointment         2
Street Light Condition                2
Litter Basket / Request               1
Unsanitary Animal Pvt Property        1
Traffic Signal Condition              1
UNSANITARY CONDITION                  1
Noise                                 1
Derelict Vehicle                      1
Missed Collection (All Materials)     1
Name: complaint_type, dtype: int64

In [45]:
data.created_date.tail()

995    2019-04-11T07:59:25.000
996    2019-04-11T07:59:26.000
997    2019-04-11T07:59:28.000
998    2019-04-11T07:59:34.000
999    2019-04-11T07:59:37.000
Name: created_date, dtype: object

In [54]:
url1 = "https://data.cityofnewyork.us/resource/fhrw-4uyv.json?$where=created_date%20between%20%272019-04-11T08:00:00%27%20and%20%272019-04-12T00:00:00%27"
data1 = pd.read_json(url1)

In [53]:
data.created_date.tail()

995    2019-04-11T07:59:25.000
996    2019-04-11T07:59:26.000
997    2019-04-11T07:59:28.000
998    2019-04-11T07:59:34.000
999    2019-04-11T07:59:37.000
Name: created_date, dtype: object

In [57]:
data1.created_date.tail()

995    2019-04-11T09:49:00.000
996    2019-04-11T09:49:00.000
997    2019-04-11T09:49:00.000
998    2019-04-11T09:49:00.000
999    2019-04-11T09:49:03.000
Name: created_date, dtype: object

In [56]:
data1.complaint_type.value_counts()

HEAT/HOT WATER                             104
Street Light Condition                      91
Illegal Parking                             73
Missed Collection (All Materials)           51
Request Large Bulky Item Collection         46
Street Condition                            46
UNSANITARY CONDITION                        34
Derelict Vehicles                           33
Blocked Driveway                            30
General Construction/Plumbing               26
Dirty Conditions                            26
Sanitation Condition                        22
Noise - Residential                         22
PAINT/PLASTER                               19
Water System                                19
Graffiti                                    17
Noise                                       14
Sidewalk Condition                          14
WATER LEAK                                  14
Derelict Vehicle                            13
ELECTRIC                                    13
DOOR/WINDOW  

In [58]:
url2 = "https://data.cityofnewyork.us/resource/fhrw-4uyv.json?$where=created_date%20between%20%272019-04-11T09:49:04%27%20and%20%272019-04-12T00:00:00%27"
data2 = pd.read_json(url2)

In [61]:
data2.created_date.tail()

995    2019-04-11T11:38:27.000
996    2019-04-11T11:38:36.000
997    2019-04-11T11:38:36.000
998    2019-04-11T11:38:36.000
999    2019-04-11T11:38:36.000
Name: created_date, dtype: object

In [65]:
url3 = "https://data.cityofnewyork.us/resource/fhrw-4uyv.json?$where=created_date%20between%20%272019-04-11T11:38:37%27%20and%20%272019-04-12T00:00:00%27"
data3 = pd.read_json(url3)

In [67]:
data3.created_date.tail()

995    2019-04-11T13:35:16.000
996    2019-04-11T13:35:24.000
997    2019-04-11T13:35:24.000
998    2019-04-11T13:36:00.000
999    2019-04-11T13:36:06.000
Name: created_date, dtype: object

In [69]:
url4 = "https://data.cityofnewyork.us/resource/fhrw-4uyv.json?$where=created_date%20between%20%272019-04-11T13:36:06%27%20and%20%272019-04-12T00:00:00%27"
data4 = pd.read_json(url4)
data4.created_date.tail()

995    2019-04-11T15:25:05.000
996    2019-04-11T15:25:09.000
997    2019-04-11T15:25:14.000
998    2019-04-11T15:25:17.000
999    2019-04-11T15:25:22.000
Name: created_date, dtype: object

In [70]:
url5 = "https://data.cityofnewyork.us/resource/fhrw-4uyv.json?$where=created_date%20between%20%272019-04-11T15:25:22%27%20and%20%272019-04-12T00:00:00%27"
data5 = pd.read_json(url5)
data5.created_date.tail()

995    2019-04-11T17:29:44.000
996    2019-04-11T17:29:44.000
997    2019-04-11T17:29:45.000
998    2019-04-11T17:29:46.000
999    2019-04-11T17:30:00.000
Name: created_date, dtype: object

In [71]:
url6 = "https://data.cityofnewyork.us/resource/fhrw-4uyv.json?$where=created_date%20between%20%272019-04-11T17:30:00%27%20and%20%272019-04-12T00:00:00%27"
data6 = pd.read_json(url6)
data6.created_date.tail()

995    2019-04-11T20:29:55.000
996    2019-04-11T20:30:00.000
997    2019-04-11T20:30:11.000
998    2019-04-11T20:30:33.000
999    2019-04-11T20:30:35.000
Name: created_date, dtype: object

In [75]:
data6.created_date[999]

'2019-04-11T20:30:35.000'

In [80]:
url7 = "https://data.cityofnewyork.us/resource/fhrw-4uyv.json?$where=created_date%20between%20%272019-04-11T20:30:35%27%20and%20%272019-04-11T23:59:48%27"
data7 = pd.read_json(url7)
data7.created_date.tail(10)

984    2019-04-11T23:57:41.000
985    2019-04-11T23:57:52.000
986    2019-04-11T23:58:00.000
987    2019-04-11T23:58:07.000
988    2019-04-11T23:58:12.000
989    2019-04-11T23:58:52.000
990    2019-04-11T23:59:00.000
991    2019-04-11T23:59:00.000
992    2019-04-11T23:59:00.000
993    2019-04-11T23:59:47.000
Name: created_date, dtype: object

In [92]:
df = pd.concat([data, data1, data2, data3, data4, data5, data6, data7], join="inner", axis=0)

In [96]:
df.head()

,address_type,agency,agency_name,bbl,borough,bridge_highway_direction,bridge_highway_name,bridge_highway_segment,city,closed_date,...,park_facility_name,resolution_action_updated_date,resolution_description,road_ramp,status,street_name,taxi_pick_up_location,unique_key,x_coordinate_state_plane,y_coordinate_state_plane
0,ADDRESS,DOHMH,Department of Health and Mental Hygiene,2.026360e+09,BRONX,NaN,NaN,NaN,BRONX,2019-04-17T17:23:06.000,...,Unspecified,2019-04-17T17:23:06.000,The Department of Health and Mental Hygiene wi...,NaN,Closed,JACKSON AVENUE,NaN,42185961,1010035.0,237583.0
1,INTERSECTION,DOHMH,Department of Health and Mental Hygiene,NaN,QUEENS,NaN,NaN,NaN,Far Rockaway,2019-04-17T00:00:00.000,...,Unspecified,2019-04-17T00:00:00.000,The Department of Health and Mental Hygiene wi...,NaN,Closed,NaN,NaN,42192716,1034103.0,158016.0
2,ADDRESS,DOHMH,Department of Health and Mental Hygiene,2.057030e+09,BRONX,NaN,NaN,NaN,BRONX,2019-04-22T00:00:00.000,...,Unspecified,2019-04-22T00:00:00.000,The Department of Health and Mental Hygiene wi...,NaN,Closed,WEST 230 STREET,NaN,42195383,1009847.0,259067.0
3,ADDRESS,DOHMH,Department of Health and Mental Hygiene,NaN,MANHATTAN,NaN,NaN,NaN,NEW YORK,2019-04-11T00:00:01.000,...,Unspecified,2019-04-22T13:05:50.000,Callers should contact the DOHMH Foodborne Ill...,NaN,Closed,EAST 24 STREET,NaN,42195875,987900.0,209335.0
4,ADDRESS,DEP,Department of Environmental Protection,4.019370e+09,QUEENS,NaN,NaN,NaN,Corona,2019-04-14T00:00:00.000,...,Unspecified,2019-04-14T00:00:00.000,The Department of Environmental Protection req...,NaN,Closed,100 STREET,NaN,42197463,1023037.0,208811.0


In [97]:
df.complaint_type.value_counts()

HEAT/HOT WATER                          710
Request Large Bulky Item Collection     617
Illegal Parking                         567
Noise - Residential                     398
Blocked Driveway                        342
Street Condition                        308
Noise                                   290
Street Light Condition                  271
UNSANITARY CONDITION                    238
General Construction/Plumbing           214
Water System                            188
PAINT/PLASTER                           171
PLUMBING                                169
Derelict Vehicles                       153
Missed Collection (All Materials)       146
Sanitation Condition                    136
Derelict Vehicle                        134
Noise - Street/Sidewalk                 122
Dirty Conditions                        121
Sidewalk Condition                      113
Building/Use                            104
Electronics Waste Appointment           102
DOOR/WINDOW                     

In [98]:
df.to_csv('311_411.csv', index=False)

In [121]:
df['complaint_type'][df['borough']=='BROOKLYN'].value_counts().head(5)

Request Large Bulky Item Collection    233
Illegal Parking                        198
HEAT/HOT WATER                         183
Blocked Driveway                       142
Noise - Residential                    105
Name: complaint_type, dtype: int64

In [122]:
df['complaint_type'][df['borough']=='QUEENS'].value_counts().head(5)

Request Large Bulky Item Collection    254
Illegal Parking                        173
Street Condition                       132
Blocked Driveway                       128
HEAT/HOT WATER                         108
Name: complaint_type, dtype: int64

In [123]:
df['complaint_type'][df['borough']=='MANHATTAN'].value_counts().head(5)

Noise                      171
HEAT/HOT WATER             157
Illegal Parking             96
Noise - Residential         92
Noise - Street/Sidewalk     65
Name: complaint_type, dtype: int64

In [124]:
df['complaint_type'][df['borough']=='BRONX'].value_counts().head(5)

HEAT/HOT WATER          258
Noise - Residential     109
UNSANITARY CONDITION     99
PAINT/PLASTER            67
Illegal Parking          64
Name: complaint_type, dtype: int64

In [125]:
df['complaint_type'][df['borough']=='STATEN ISLAND'].value_counts().head(5)

Request Large Bulky Item Collection    45
Illegal Parking                        32
Street Condition                       29
Electronics Waste Appointment          28
Missed Collection (All Materials)      23
Name: complaint_type, dtype: int64

In [127]:
df['agency'][df['borough']=='BROOKLYN'].value_counts().head(5)

NYPD    587
DSNY    572
HPD     531
DOT     231
DOB     203
Name: agency, dtype: int64

In [129]:
df['agency'][df['borough']=='QUEENS'].value_counts().head(5)

NYPD    519
DSNY    443
DOT     315
HPD     233
DOB     151
Name: agency, dtype: int64

In [130]:
df['agency'][df['borough']=='MANHATTAN'].value_counts().head(5)

HPD     371
NYPD    361
DEP     245
DOT     155
DOB     125
Name: agency, dtype: int64

In [131]:
df['agency'][df['borough']=='BRONX'].value_counts().head(5)

HPD     652
NYPD    267
DOT     124
DSNY    112
DOB      66
Name: agency, dtype: int64

In [132]:
df['agency'][df['borough']=='BROOKLYN'].value_counts().head(5)

NYPD    587
DSNY    572
HPD     531
DOT     231
DOB     203
Name: agency, dtype: int64

Function

In [19]:
df = pd.read_csv('311_411.csv')

In [5]:
df.agency_name.value_counts()

Department of Housing Preservation and Development    1805
New York City Police Department                       1798
Department of Transportation                           910
Department of Sanitation                               763
Department of Environmental Protection                 662
Department of Buildings                                567
Department of Parks and Recreation                     223
Department of Health and Mental Hygiene                208
BCC - Brooklyn South                                   112
BCC - Brooklyn North                                    76
Taxi and Limousine Commission                           73
BCC - Staten Island                                     50
Operations Unit - Department of Homeless Services       42
Department of Consumer Affairs                          42
BCC - Queens East                                       39
Personal Exemption Unit                                 37
A - Brooklyn                                            

In [6]:
df.columns

Index(['address_type', 'agency', 'agency_name', 'bbl', 'borough',
       'bridge_highway_direction', 'bridge_highway_name',
       'bridge_highway_segment', 'city', 'closed_date', 'community_board',
       'complaint_type', 'created_date', 'cross_street_1', 'cross_street_2',
       'descriptor', 'due_date', 'facility_type', 'incident_address',
       'incident_zip', 'intersection_street_1', 'intersection_street_2',
       'latitude', 'location', 'location_type', 'longitude',
       'open_data_channel_type', 'park_borough', 'park_facility_name',
       'resolution_action_updated_date', 'resolution_description', 'road_ramp',
       'status', 'street_name', 'taxi_pick_up_location', 'unique_key',
       'x_coordinate_state_plane', 'y_coordinate_state_plane'],
      dtype='object')

In [9]:
df.resolution_description.value_counts()

The Police Department responded to the complaint and with the information available observed no evidence of the violation at that time.                                                                                                                                                                                                                 574
The Department of Housing Preservation and Development inspected the following conditions. No violations were issued. The complaint has been closed.                                                                                                                                                                                                    497
Service Request status for this request is available on the Department of Transportationâs website. Please click the âLearn Moreâ link below.                                                                                                                                                             

In [11]:
df.loc[df['resolution_description']=="The Police Department responded to the complaint and with the information available observed no evidence of the violation at that time."]

,address_type,agency,agency_name,bbl,borough,bridge_highway_direction,bridge_highway_name,bridge_highway_segment,city,closed_date,...,park_facility_name,resolution_action_updated_date,resolution_description,road_ramp,status,street_name,taxi_pick_up_location,unique_key,x_coordinate_state_plane,y_coordinate_state_plane
124,ADDRESS,NYPD,New York City Police Department,3.030640e+09,BROOKLYN,NaN,NaN,NaN,BROOKLYN,2019-04-11T00:50:30.000,...,Unspecified,2019-04-11T00:50:31.000,The Police Department responded to the complai...,NaN,Closed,BUSHWICK AVENUE,NaN,42185372,1000947.0,196992.0
137,ADDRESS,NYPD,New York City Police Department,3.030930e+09,BROOKLYN,NaN,NaN,NaN,BROOKLYN,2019-04-11T01:29:11.000,...,Unspecified,2019-04-11T01:29:11.000,The Police Department responded to the complai...,NaN,Closed,HARRISON PLACE,NaN,42184252,1002873.0,196570.0
145,ADDRESS,NYPD,New York City Police Department,3.053240e+09,BROOKLYN,NaN,NaN,NaN,BROOKLYN,2019-04-11T01:06:45.000,...,Unspecified,2019-04-11T01:06:45.000,The Police Department responded to the complai...,NaN,Closed,CATON AVENUE,NaN,42191105,989792.0,175113.0
151,ADDRESS,NYPD,New York City Police Department,4.017750e+09,QUEENS,NaN,NaN,NaN,CORONA,2019-04-11T04:06:39.000,...,Unspecified,2019-04-11T04:06:39.000,The Police Department responded to the complai...,NaN,Closed,108 STREET,NaN,42192434,1023175.0,213338.0
157,ADDRESS,NYPD,New York City Police Department,3.017660e+09,BROOKLYN,NaN,NaN,NaN,BROOKLYN,2019-04-11T02:31:19.000,...,Unspecified,2019-04-11T02:31:19.000,The Police Department responded to the complai...,NaN,Closed,WILLOUGHBY AVENUE,NaN,42185083,997644.0,191968.0
168,ADDRESS,NYPD,New York City Police Department,4.031980e+09,QUEENS,NaN,NaN,NaN,FOREST HILLS,2019-04-11T07:33:37.000,...,Unspecified,2019-04-11T07:33:38.000,The Police Department responded to the complai...,NaN,Closed,JUNO STREET,NaN,42187623,1024687.0,200142.0
169,ADDRESS,NYPD,New York City Police Department,1.020310e+09,MANHATTAN,NaN,NaN,NaN,NEW YORK,2019-04-11T07:21:28.000,...,Unspecified,2019-04-11T07:21:28.000,The Police Department responded to the complai...,NaN,Closed,WEST 146 STREET,NaN,42192276,1000835.0,239098.0
172,ADDRESS,NYPD,New York City Police Department,4.017160e+09,QUEENS,NaN,NaN,NaN,CORONA,2019-04-11T02:41:30.000,...,Unspecified,2019-04-11T02:41:30.000,The Police Department responded to the complai...,NaN,Closed,NORTHERN BOULEVARD,NaN,42192399,1021009.0,215264.0
179,ADDRESS,NYPD,New York City Police Department,1.020310e+09,MANHATTAN,NaN,NaN,NaN,NEW YORK,2019-04-11T07:20:26.000,...,Unspecified,2019-04-11T07:20:26.000,The Police Department responded to the complai...,NaN,Closed,WEST 146 STREET,NaN,42187663,1000835.0,239098.0
183,ADDRESS,NYPD,New York City Police Department,4.071870e+09,QUEENS,NaN,NaN,NaN,FRESH MEADOWS,2019-04-11T05:35:38.000,...,Unspecified,2019-04-11T05:35:38.000,The Police Department responded to the complai...,NaN,Closed,UTOPIA PARKWAY,NaN,42191112,1041701.0,204636.0


In [ ]:
df['resolution_description'].contains

In [14]:
df.loc[df['agency']=='DOT']['resolution_description'].value_counts()

Service Request status for this request is available on the Department of Transportationâs website. Please click the âLearn Moreâ link below.                                                                                                                                                                                                                                                                                                                                    351
The Department of Transportation inspected this complaint and repaired the problem.                                                                                                                                                                                                                                                                                                                                                                                                    120
The Department of Transportation inspected the con

In [21]:
df['complaint_type']

0                                    Rodent
1                            Standing Water
2                            Standing Water
3                            Food Poisoning
4                                     Noise
5                            Food Poisoning
6                            Food Poisoning
7                            Food Poisoning
8                            Standing Water
9                            Standing Water
10                                   Rodent
11                           Food Poisoning
12                           Standing Water
13                           Food Poisoning
14                           Food Poisoning
15                                   Rodent
16           Unsanitary Animal Pvt Property
17                                   Rodent
18                                   Rodent
19                                   Rodent
20                                   Rodent
21                                   Rodent
22                              

In [26]:
df_agency = df[['agency', 'agency_name']]

In [30]:
df_agency.loc[df['agency']=='DOITT']

,agency,agency_name


In [31]:
df[df['agency']=='NYPD']

,address_type,agency,agency_name,bbl,borough,bridge_highway_direction,bridge_highway_name,bridge_highway_segment,city,closed_date,...,park_facility_name,resolution_action_updated_date,resolution_description,road_ramp,status,street_name,taxi_pick_up_location,unique_key,x_coordinate_state_plane,y_coordinate_state_plane
123,ADDRESS,NYPD,New York City Police Department,1.004190e+09,MANHATTAN,NaN,NaN,NaN,NEW YORK,2019-04-11T02:52:39.000,...,Unspecified,2019-04-11T02:52:39.000,The Police Department responded to the complai...,NaN,Closed,ELDRIDGE STREET,NaN,42187601,986614.0,201279.0
124,ADDRESS,NYPD,New York City Police Department,3.030640e+09,BROOKLYN,NaN,NaN,NaN,BROOKLYN,2019-04-11T00:50:30.000,...,Unspecified,2019-04-11T00:50:31.000,The Police Department responded to the complai...,NaN,Closed,BUSHWICK AVENUE,NaN,42185372,1000947.0,196992.0
134,ADDRESS,NYPD,New York City Police Department,5.062490e+09,STATEN ISLAND,NaN,NaN,NaN,STATEN ISLAND,2019-04-11T01:50:35.000,...,Unspecified,2019-04-11T01:50:35.000,The Police Department responded and upon arriv...,NaN,Closed,ELY AVENUE,NaN,42186049,934261.0,135443.0
137,ADDRESS,NYPD,New York City Police Department,3.030930e+09,BROOKLYN,NaN,NaN,NaN,BROOKLYN,2019-04-11T01:29:11.000,...,Unspecified,2019-04-11T01:29:11.000,The Police Department responded to the complai...,NaN,Closed,HARRISON PLACE,NaN,42184252,1002873.0,196570.0
138,ADDRESS,NYPD,New York City Police Department,3.051050e+09,BROOKLYN,NaN,NaN,NaN,BROOKLYN,2019-04-11T03:03:48.000,...,Unspecified,2019-04-11T03:03:48.000,The Police Department responded to the complai...,NaN,Closed,ERASMUS STREET,NaN,42184280,998197.0,176012.0
139,LATLONG,NYPD,New York City Police Department,NaN,STATEN ISLAND,NaN,NaN,NaN,STATEN ISLAND,2019-04-11T04:55:05.000,...,Unspecified,2019-04-11T04:55:05.000,The Police Department responded and upon arriv...,NaN,Closed,BENNETT AVENUE,NaN,42185997,934152.0,135576.0
140,ADDRESS,NYPD,New York City Police Department,3.007780e+09,BROOKLYN,NaN,NaN,NaN,BROOKLYN,2019-04-11T02:46:34.000,...,Unspecified,2019-04-11T02:46:34.000,The Police Department responded to the complai...,NaN,Closed,49 STREET,NaN,42185437,983590.0,172896.0
141,ADDRESS,NYPD,New York City Police Department,1.008610e+09,MANHATTAN,NaN,NaN,NaN,NEW YORK,2019-04-11T02:40:29.000,...,Unspecified,2019-04-11T02:40:29.000,The Police Department responded to the complai...,NaN,Closed,5 AVENUE,NaN,42188820,988208.0,211371.0
142,ADDRESS,NYPD,New York City Police Department,4.039168e+09,QUEENS,NaN,NaN,NaN,COLLEGE POINT,2019-04-11T01:53:40.000,...,Unspecified,2019-04-11T01:53:40.000,The Police Department responded and upon arriv...,NaN,Closed,121 STREET,NaN,42189484,1026764.0,228596.0
145,ADDRESS,NYPD,New York City Police Department,3.053240e+09,BROOKLYN,NaN,NaN,NaN,BROOKLYN,2019-04-11T01:06:45.000,...,Unspecified,2019-04-11T01:06:45.000,The Police Department responded to the complai...,NaN,Closed,CATON AVENUE,NaN,42191105,989792.0,175113.0


In [33]:
df[df['agency']=='NYPD']

,address_type,agency,agency_name,bbl,borough,bridge_highway_direction,bridge_highway_name,bridge_highway_segment,city,closed_date,...,park_facility_name,resolution_action_updated_date,resolution_description,road_ramp,status,street_name,taxi_pick_up_location,unique_key,x_coordinate_state_plane,y_coordinate_state_plane
123,ADDRESS,NYPD,New York City Police Department,1.004190e+09,MANHATTAN,NaN,NaN,NaN,NEW YORK,2019-04-11T02:52:39.000,...,Unspecified,2019-04-11T02:52:39.000,The Police Department responded to the complai...,NaN,Closed,ELDRIDGE STREET,NaN,42187601,986614.0,201279.0
124,ADDRESS,NYPD,New York City Police Department,3.030640e+09,BROOKLYN,NaN,NaN,NaN,BROOKLYN,2019-04-11T00:50:30.000,...,Unspecified,2019-04-11T00:50:31.000,The Police Department responded to the complai...,NaN,Closed,BUSHWICK AVENUE,NaN,42185372,1000947.0,196992.0
134,ADDRESS,NYPD,New York City Police Department,5.062490e+09,STATEN ISLAND,NaN,NaN,NaN,STATEN ISLAND,2019-04-11T01:50:35.000,...,Unspecified,2019-04-11T01:50:35.000,The Police Department responded and upon arriv...,NaN,Closed,ELY AVENUE,NaN,42186049,934261.0,135443.0
137,ADDRESS,NYPD,New York City Police Department,3.030930e+09,BROOKLYN,NaN,NaN,NaN,BROOKLYN,2019-04-11T01:29:11.000,...,Unspecified,2019-04-11T01:29:11.000,The Police Department responded to the complai...,NaN,Closed,HARRISON PLACE,NaN,42184252,1002873.0,196570.0
138,ADDRESS,NYPD,New York City Police Department,3.051050e+09,BROOKLYN,NaN,NaN,NaN,BROOKLYN,2019-04-11T03:03:48.000,...,Unspecified,2019-04-11T03:03:48.000,The Police Department responded to the complai...,NaN,Closed,ERASMUS STREET,NaN,42184280,998197.0,176012.0
139,LATLONG,NYPD,New York City Police Department,NaN,STATEN ISLAND,NaN,NaN,NaN,STATEN ISLAND,2019-04-11T04:55:05.000,...,Unspecified,2019-04-11T04:55:05.000,The Police Department responded and upon arriv...,NaN,Closed,BENNETT AVENUE,NaN,42185997,934152.0,135576.0
140,ADDRESS,NYPD,New York City Police Department,3.007780e+09,BROOKLYN,NaN,NaN,NaN,BROOKLYN,2019-04-11T02:46:34.000,...,Unspecified,2019-04-11T02:46:34.000,The Police Department responded to the complai...,NaN,Closed,49 STREET,NaN,42185437,983590.0,172896.0
141,ADDRESS,NYPD,New York City Police Department,1.008610e+09,MANHATTAN,NaN,NaN,NaN,NEW YORK,2019-04-11T02:40:29.000,...,Unspecified,2019-04-11T02:40:29.000,The Police Department responded to the complai...,NaN,Closed,5 AVENUE,NaN,42188820,988208.0,211371.0
142,ADDRESS,NYPD,New York City Police Department,4.039168e+09,QUEENS,NaN,NaN,NaN,COLLEGE POINT,2019-04-11T01:53:40.000,...,Unspecified,2019-04-11T01:53:40.000,The Police Department responded and upon arriv...,NaN,Closed,121 STREET,NaN,42189484,1026764.0,228596.0
145,ADDRESS,NYPD,New York City Police Department,3.053240e+09,BROOKLYN,NaN,NaN,NaN,BROOKLYN,2019-04-11T01:06:45.000,...,Unspecified,2019-04-11T01:06:45.000,The Police Department responded to the complai...,NaN,Closed,CATON AVENUE,NaN,42191105,989792.0,175113.0
